In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [ ]:
# Youtube API

import os

# ใส่ API Key ของคุณ
API_KEY = os.getenv("CLIENT_ID_YOUTUBE")

In [31]:
## test ดึงรายชื่อแบบเดี่ยวๆ

from googleapiclient.discovery import build
import csv
import os

# ใส่ API Key ของคุณ
API_KEY = os.getenv("CLIENT_ID_YOUTUBE")
youtube = build("youtube", "v3", developerKey=API_KEY)

def get_bts_official_mv(query, max_results=50):
    # ค้นหาเพลงของ BTS ที่เป็น Official MV
    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=max_results
    )
    response = request.execute()

    # เก็บข้อมูลวิดีโอ
    video_ids = [item["id"]["videoId"] for item in response["items"]]
    video_details = youtube.videos().list(
        part="snippet,statistics",
        id=",".join(video_ids)
    ).execute()

    results = []
    for item in video_details["items"]:
        title = item["snippet"]["title"]
        channel_title = item["snippet"]["channelTitle"]  # ชื่อศิลปินหรือแชนแนล
        publish_date = item["snippet"]["publishedAt"]
        view_count = item["statistics"].get("viewCount", 0)
        like_count = item["statistics"].get("likeCount", 0)
        video_url = f"https://www.youtube.com/watch?v={item['id']}"  # ลิ้งวิดีโอ

        # กรองเฉพาะวิดีโอที่มีคำว่า "Official MV" ในชื่อ
        if "Official MV" in title and "BTS" in title:
            results.append({
                "title": title,
                "artist": channel_title,
                "publish_date": publish_date,
                "views": int(view_count),
                "likes": int(like_count),
                "video_url": video_url,
            })
    return results

# ค้นหาเพลง BTS
query = "BTS Official MV"
bts_songs = get_bts_official_mv(query, max_results=50)

# บันทึกลงไฟล์ CSV
with open("bts_official_mv.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["title", "artist", "publish_date", "views", "likes", "video_url"])
    writer.writeheader()
    writer.writerows(bts_songs)

print("Data saved to bts_official_mv.csv")


Data saved to bts_official_mv.csv


In [ ]:
from googleapiclient.discovery import build
import csv
import pandas as pd
import time
import os

# ใส่ API Key ของคุณ
API_KEY = os.getenv("CLIENT_ID_YOUTUBE")
youtube = build("youtube", "v3", developerKey=API_KEY)

QUOTA_LIMIT = 10000  # โควต้าสูงสุดต่อวัน
QUOTA_USED = 0       # โควต้าที่ใช้ไป

def check_quota_cost(cost):
    global QUOTA_USED, QUOTA_LIMIT
    if QUOTA_USED + cost > QUOTA_LIMIT:
        return False  # โควต้าไม่เพียงพอ
    QUOTA_USED += cost
    return True  # ใช้งานได้

def get_video_details(query, max_results=1):
    global QUOTA_USED

    # ตรวจสอบว่าโควต้าพอสำหรับการค้นหา
    if not check_quota_cost(100):  # `search.list` ใช้ 100 หน่วย
        return None

    # ค้นหาเพลงใน YouTube
    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        maxResults=max_results
    )
    response = request.execute()

    if not response["items"]:
        return None  # ไม่มีผลลัพธ์

    video_id = response["items"][0]["id"]["videoId"]

    # ตรวจสอบว่าโควต้าพอสำหรับการดึงข้อมูลวิดีโอ
    if not check_quota_cost(1):  # `videos.list` ใช้ 1 หน่วยต่อวิดีโอ
        return None

    video_details = youtube.videos().list(
        part="snippet,statistics",
        id=video_id
    ).execute()["items"][0]

    # ดึงข้อมูลวิดีโอ
    title = video_details["snippet"]["title"]
    channel_title = video_details["snippet"]["channelTitle"]
    publish_date = video_details["snippet"]["publishedAt"]
    view_count = video_details["statistics"].get("viewCount", 0)
    like_count = video_details["statistics"].get("likeCount", 0)
    video_url = f"https://www.youtube.com/watch?v={video_id}"

    return {
        "youtube_title": title,
        "youtube_artist": channel_title,
        "publish_date": publish_date,
        "views": int(view_count),
        "likes": int(like_count),
        "video_url": video_url,
    }

# อ่านข้อมูลจาก spotify-2023.csv
# กำหนดคอลัมเพลงดีๆ
spotify_data = pd.read_csv("spotify-2023.csv", encoding='ISO-8859-1')
track_names = spotify_data["track_name"].tolist()  # ตรวจสอบว่าคอลัมน์นี้มีชื่อเพลง

# เก็บผลลัพธ์
results = []
for track in track_names:
    if not check_quota_cost(0):  # ตรวจสอบว่ามีโควต้าเหลือหรือไม่
        print("Quota limit reached. Stopping execution.")
        break
    video_info = get_video_details(track)
    if video_info:
        results.append({
            "spotify_track_name": track,
            **video_info
        })
    else:
        results.append({
            "spotify_track_name": track,
            "youtube_title": None,
            "youtube_artist": None,
            "publish_date": None,
            "views": None,
            "likes": None,
            "video_url": None,
        })

# บันทึกผลลัพธ์ลงไฟล์ CSV
with open("spotify_youtube_comparison.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=[
        "spotify_track_name", "youtube_title", "youtube_artist", "publish_date", "views", "likes", "video_url"
    ])
    writer.writeheader()
    writer.writerows(results)

print("Data saved to spotify_youtube_comparison.csv")


code to match Spotify song and Youtube song

In [ ]:
import re
from googleapiclient.discovery import build
import csv
import os

# ใส่ API Key ของคุณ
API_KEY = os.getenv("YOUTUBE_API_KEY5")
youtube = build("youtube", "v3", developerKey=API_KEY)

not_matched_results = []

def get_thai_song_mv(query, artist_name, track_name,song, max_results=10):
    try:
        request = youtube.search().list(
            q=query,
            part="snippet",
            type="video",
            maxResults=max_results
        )
        response = request.execute()

        video_ids = [item["id"]["videoId"] for item in response["items"] if "videoId" in item["id"]]
        video_details = youtube.videos().list(
            part="snippet,statistics",
            id=",".join(video_ids)
        ).execute()

        results = []
        for item in video_details["items"]:
            title = item["snippet"]["title"]
            channel_title = item["snippet"]["channelTitle"]
            publish_date = item["snippet"]["publishedAt"]
            view_count = item["statistics"].get("viewCount", 0)
            like_count = item["statistics"].get("likeCount", 0)
            video_url = f"https://www.youtube.com/watch?v={item['id']}"

            # กรองเฉพาะวิดีโอที่มีคำว่า "Official" หรือ "M/V"
            # if "Official" in title or "M/V" in title:
            # ลบช่องว่างและเครื่องหมายพิเศษ
            clean_artist_name = re.sub(r'[^a-zA-Z0-9ก-๙]', '', artist_name.lower())  # รองรับทั้งภาษาอังกฤษและภาษาไทย

            clean_track_name = re.sub(r'[^a-zA-Z0-9ก-๙]', '', track_name.lower())  # รองรับทั้งภาษาอังกฤษและภาษาไทย
            clean_title_YT = re.sub(r'[^a-zA-Z0-9ก-๙]', '', title.lower())

            if len(clean_track_name) > len(clean_title_YT):
                if  clean_title_YT in clean_track_name and (clean_artist_name not in clean_title_YT and clean_title_YT in clean_track_name or clean_artist_name in clean_title_YT):
                    results.append({
                            "title": title,
                            "artist": channel_title,
                            "publish_date": publish_date,
                            "views": int(view_count),
                            "likes": int(like_count),
                            "video_url": video_url,
                    })
            else:
                # ตรวจสอบว่า track_name ตรงกับ title ใน YouTube หรือไม่             
                if  clean_track_name in clean_title_YT and (clean_artist_name not in clean_title_YT and clean_track_name in clean_title_YT or clean_artist_name in clean_title_YT):
                    results.append({
                            "title": title,
                            "artist": channel_title,
                            "publish_date": publish_date,
                            "views": int(view_count),
                            "likes": int(like_count),
                            "video_url": video_url,
                    })
        
        print(results)

        # เรียงลำดับผลลัพธ์ตามยอดวิว (จากมากไปน้อย)
        results_sorted = sorted(results, key=lambda x: x["views"], reverse=True)

        # หากต้องการเลือกเพลงที่มียอดวิวสูงสุดเพียง 1 เพลง
        # return results_sorted[:1]  # เลือกเพลงที่มียอดวิวสูงสุด 1 เพลง

        return results_sorted[:1]  # หากต้องการผลลัพธ์ทั้งหมดที่เรียงตามยอดวิว

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        not_matched_results.append(song)
        return []


# ฟังก์ชันที่ใช้ในการ match ข้อมูลจาก CSV กับ YouTube
def match_songs_from_csv(csv_file):
    with open(csv_file, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        songs = list(reader)

    results = []
    for song in songs:
        artist_name = song["artist_names"]
        track_name = song["track_name"]
        source = song["source"]
        query = f"{artist_name} {track_name}"

        # ค้นหาเพลงใน YouTube
        youtube_results = get_thai_song_mv(query,artist_name, track_name,song)
        print(youtube_results)


        if youtube_results:
            for result in youtube_results:
                # เปรียบเทียบชื่อศิลปิน โดยไม่สนใจตัวพิมพ์ใหญ่และช่องว่าง
                youtube_artist = result["artist"].lower().replace(" ", "")
                csv_artist = artist_name.lower().replace(" ", "")

                # ลบช่องว่างและเครื่องหมายพิเศษ
                clean_track_names = re.sub(r'[^a-zA-Z0-9ก-๙]', '', track_name.lower())  # รองรับทั้งภาษาอังกฤษและภาษาไทย
                clean_test_play = re.sub(r'[^a-zA-Z0-9ก-๙]', '', result["title"].lower())

                if len(clean_track_names) > len(clean_test_play):
                    if clean_test_play in clean_track_names:

                        results.append({ 
                            "youtube_title": result["title"],
                            "youtube_artist": result["artist"],
                            "publish_date": result["publish_date"],
                            "views": result["views"],
                            "likes": result["likes"],
                            "video_url": result["video_url"],
                            "youtube_source": result["artist"],  # ช่อง YouTube อาจจะเป็นแหล่งที่มาของเพลง
                        })
                else:
                    # ตรวจสอบว่า track_name ตรงกับ title ใน YouTube หรือไม่
                    if clean_track_names in clean_test_play:

                            results.append({ 
                                "youtube_title": result["title"],
                                "youtube_artist": result["artist"],
                                "publish_date": result["publish_date"],
                                "views": result["views"],
                                "likes": result["likes"],
                                "video_url": result["video_url"],
                                "youtube_source": result["artist"],  # ช่อง YouTube อาจจะเป็นแหล่งที่มาของเพลง
                            })
        else:
             not_matched_results.append(song)

    with open("matchsong.csv", "a", newline="", encoding="utf-8") as match_file, \
     open("emptynotmatch.csv", "a", newline="", encoding="utf-8") as not_match_file:

        match_fieldnames = [
            "youtube_title",
            "youtube_artist",
            "publish_date",
            "views",
            "likes",
            "video_url",
            "youtube_source",
        ]
        not_match_fieldnames = songs[0].keys()  # ใช้คอลัมน์เดิมจากไฟล์ต้นฉบับ

        # สร้าง writer สำหรับ match_file
        match_writer = csv.DictWriter(match_file, fieldnames=match_fieldnames)
        if match_file.tell() == 0:  # ถ้าไฟล์ยังว่าง
            match_writer.writeheader()

        # สร้าง writer สำหรับ not_match_file
        not_match_writer = csv.DictWriter(not_match_file, fieldnames=not_match_fieldnames)
        if not_match_file.tell() == 0:  # ถ้าไฟล์ยังว่าง
            not_match_writer.writeheader()

        # เขียนข้อมูลที่ match ลงไฟล์ matchsong.csv
        match_writer.writerows(results)

        # เขียนข้อมูลที่ไม่ match ลงไฟล์ emptynotmatch.csv
        # ลบรายการที่ซ้ำกันใน not_matched_results
        unique_array = [dict(t) for t in {tuple(d.items()) for d in not_matched_results}]
        not_match_writer.writerows(unique_array)


    print("Data saved to matched_songs.csv")
    print(len(results))
    print("NotMatch" , unique_array)

# เรียกใช้งานฟังก์ชัน
match_songs_from_csv("test.csv")  # เปลี่ยนเป็นชื่อไฟล์ CSV ที่มีข้อมูลเพลง

Code test Re

In [ ]:
# บันทึกข้อมูลลงในไฟล์ CSV
    # with open("matched_songs12.csv", "w", newline="", encoding="utf-8") as file:
    #     fieldnames = [
    #         "youtube_title", 
    #         "youtube_artist", 
    #         "publish_date", 
    #         "views", 
    #         "likes", 
    #         "video_url", 
    #         "youtube_source"
    #     ]
    #     writer = csv.DictWriter(file, fieldnames=fieldnames)
    #     writer.writeheader()
    #     writer.writerows(results)
    # with open("matchsong.csv", "a", newline="", encoding="utf-8") as match_file, \
    #      open("emptynotmatch.csv", "a", newline="", encoding="utf-8") as not_match_file:

    #     # ตรวจสอบว่าไฟล์มี header แล้วหรือยัง
    #     if match_file.tell() == 0:  # ถ้าไฟล์ยังว่าง
    #         match_fieldnames = [
    #             "youtube_title",
    #             "youtube_artist",
    #             "publish_date",
    #             "views",
    #             "likes",
    #             "video_url",
    #             "youtube_source",
    #         ]
    #         match_writer = csv.DictWriter(match_file, fieldnames=match_fieldnames)
    #         match_writer.writeheader()

    #     if not_match_file.tell() == 0:  # ถ้าไฟล์ยังว่าง
    #         not_match_fieldnames = songs[0].keys()  # ใช้คอลัมน์เดิมจากไฟล์ต้นฉบับ
    #         not_match_writer = csv.DictWriter(not_match_file, fieldnames=not_match_fieldnames)
    #         not_match_writer.writeheader()

    #     # เขียนข้อมูลที่ match ลงไฟล์ matchsong.csv
    #     match_writer.writerows(results)

    #     # เขียนข้อมูลที่ไม่ match ลงไฟล์ emptynotmatch.csv
    #     not_match_writer.writerows(not_matched_results)

In [49]:
import re

track_names = 'Sunflower - Spider-Man: Into the Spider-Verse'
test_play = 'Post Malone, Swae Lee - Sunflower (Spider-Man: Into the Spider-Verse) (Official Video)'

# ลบช่องว่างและเครื่องหมายพิเศษ
clean_track_names = re.sub(r'[^a-zA-Z0-9ก-๙]', '', track_names.lower())  # รองรับทั้งภาษาอังกฤษและภาษาไทย
clean_test_play = re.sub(r'[^a-zA-Z0-9ก-๙]', '', test_play.lower()) 

if clean_track_names in clean_test_play:
    print("True")

print(clean_track_names)  # จะได้ผลลัพธ์ 'sunflowerspidermanintothespiderverse'
print(clean_test_play) 

True
sunflowerspidermanintothespiderverse
postmaloneswaeleesunflowerspidermanintothespiderverseofficialvideo


In [48]:
import re

track_names = 'เป็นไรไหม ?'
test_play = 'เป็นไรไหม?'

clean_track_names = re.sub(r'[^a-zA-Z0-9ก-๙]', '', track_names.lower())  # รองรับทั้งภาษาอังกฤษและภาษาไทย
clean_test_play = re.sub(r'[^a-zA-Z0-9ก-๙]', '', test_play.lower()) 

if clean_track_names in clean_test_play:
    print("True")

print(clean_track_names)  # จะได้ผลลัพธ์ 'sunflowerspidermanintothespiderverse'
print(clean_test_play) 

True
เป็นไรไหม
เป็นไรไหม
